In [47]:
from arcgis.gis import GIS
import datetime

"""accounts to keep from user/groups/items deletion"""
ignore_accounts_online = ['DavidJVitale', 'yjiang_geosaurus', 'amani_geosaurus', 'api_data_owner',
                   'bmajor_geosaurus', 'rsingh_geosaurus', 'rohitgeo', 'andrew887',
                   'cwhitmore_geosaurus', 'ArcGISPyAPIBot']

ignore_accounts_playground = ['andrew', 'andrew.chapkowski', 'dvitale', 'david.vitale',
                   'atma.mani', 'john.yaist', 'bill.major', 'YJiang',
                   'rohit.singh', 'rohitgeo', 'gbochenek_python',
                   'system_publisher', 'admin', 'portaladmin',
                   'Demo_User', 'First_User', 'Second_User',
                   'api_data_owner', 'arcgis_python', 'temp_execution']

"""accounts that you want to delete groups and items, but keep user"""
target_accounts_online = ['arcgis_python']
target_accounts_playground = ['arcgis_python']

"""data to publish"""
data_paths = [r'\\archive\crdata\Geosaurus_datasets\data_prep\csv\Trailheads.csv']

"""create GIS connection via admin credentials"""
# gis_online = GIS(profile="your_online_admin_profile")
# gis_playground = GIS(profile='your_ent_admin_profile')
# gis_online = GIS("https://geosaurus.maps.arcgis.com", "ArcGISPyAPIBot", "geosaurus_automation123", profile="your_online_admin_profile")
# gis_online = GIS("https://geosaurus.maps.arcgis.com", "yjiang_geosaurus", "Cody0924?")

In [48]:
def delete_depending_items(dependent_item):
    """deletes the item's depending items, and then the item"""
    depending_items = None
    try:
        depending_items = dependent_item.dependent_to()
        if depending_items['list']:
            for item in depending_items['list']:
                delete_depending_items(item)
    except:
        print("=== could not get item list %s" % dependent_item.homepage)
        
    if dependent_item.protected:
        dependent_item.protect(False)
    try:
        print("=== deleting the item: %s" % dependent_item.homepage)
        dependent_item.delete()
    except:
        print("=== could not delete non-dependent item %s" % dependent_item.homepage)


def delete_items(user):
    """deletes the user items"""
    folders = [None] + user.folders
    for folder in folders:
        print("=== deleting inside folder: %s" % folder)
        for item in user.items(folder=folder, max_items=255):
            delete_depending_items(item)
    print("=== finished deleting items owned by " + user.username)


def delete_groups(gis, user):
    """deletes the user groups, and removes user from groups where user is a member of"""
    groups_for_deletion = gis.groups.get('query=owner:' + user.username)
    if groups_for_deletion is not None:
        for group in groups_for_deletion:
            try:
                print("=== deleting group %s" % group.groupid)
                group.delete()
            except:
                print("=== could not delete group %s" % group.groupid)

    for grp in user.groups:
        print("=== removing from group: %s" % grp.id)
        if grp.owner != user.username:
            try:
                grp.remove_users([user.username])
            except:
                print("=== User cannot be removed from group: %s" % grp.id)
    print("=== finished deleting groups owned by " + user.username)


def delete_for_users(gis, ignore_accounts, target_accounts):
    """deletes items and groups for users in target_accounts, and ignore others"""
    for user in gis.users.search():
        if user.username not in ignore_accounts and not user.username.startswith("esri_"):
            print("-*-*-*-*-*-*-Delete groups & items & user for %s -*-*-*-*-*-" % user.username)
            delete_items(user)
            delete_groups(gis, user)
            try:
                user.delete()
            except:
                print("could not delete user %s" % user.username)

        elif user.username in target_accounts:
            print("-*-*-*-*-*-*-Delete groups & items for %s -*-*-*-*-*-*-*-*-" % user.username)
            delete_items(user)
            delete_groups(gis, user)

        else:
            print("-*-*-*-*-*-*-*-*-No Delete for %s -*-*-*-*-*-*-*-*-*-*-" % user.username)

def publish_data(gis, paths):
    """publish sample data"""
    for path in paths:
        item = gis.content.add({}, path)
        item.share(everyone=True)
        lyr = item.publish()

In [49]:
# delete_for_users(gis_online, ignore_accounts_online, target_accounts_online)
# delete_for_users(gis_playground, ignore_accounts_playground, target_accounts_playground)

In [52]:
publish_data(gis_online, data_paths)

<Item title:"Trailheads" type:CSV owner:yjiang_geosaurus>
<Item title:"Trailheads" type:Feature Layer Collection owner:yjiang_geosaurus>


In [43]:
# csv_file = './data/LA_Hub_Datasets/Trailheads.csv'
csv_file = r'\\archive\crdata\Geosaurus_datasets\data_prep\csv\Trailheads.csv'
csv_item = gis_online.content.add({}, csv_file)

In [44]:
csv_item

<Item title:"Trailheads" type:CSV owner:ArcGISPyAPIBot>

In [46]:
csv_lyr = csv_item.publish(None)

KeyError: 'error'

In [18]:
csv_lyr

<Item title:"Trailheads" type:Feature Layer Collection owner:yjiang_geosaurus>